In [1]:
## TOPIC Modelling

In [2]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import Dataset
df= pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df = df.drop(['rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1)

In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [6]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [7]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [8]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [9]:
## BASE MODEL



# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11455.377931215236
Perplexity:  1866.1054411160621
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [11]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [12]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))



Best Model's Params:  {'learning_decay': 0.5, 'n_components': 5}
Best Log Likelihood Score:  -2451.042542183328
Model Perplexity:  915.7262805269982


In [13]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.380000,0.030000,0.280000,0.030000,0.280000,0
Doc1,0.030000,0.030000,0.030000,0.030000,0.870000,4
Doc2,0.050000,0.050000,0.050000,0.050000,0.810000,4
Doc3,0.550000,0.310000,0.050000,0.040000,0.050000,0
Doc4,0.040000,0.040000,0.040000,0.040000,0.860000,4
Doc5,0.030000,0.870000,0.030000,0.030000,0.030000,1
Doc6,0.360000,0.030000,0.030000,0.030000,0.550000,4
Doc7,0.040000,0.400000,0.040000,0.040000,0.480000,4
Doc8,0.050000,0.840000,0.040000,0.040000,0.040000,1
Doc9,0.030000,0.880000,0.030000,0.030000,0.030000,1


In [14]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  49.731719
0       51.163364 -159.968018       2        1  21.721641
1      -73.144554   21.523815       3        1  14.545768
2      -66.410133 -110.606438       4        1  10.310689
3       54.417324   12.443444       5        1   3.690183, topic_info=                       Term       Freq      Total Category  logprob  loglift
72               corruption  15.000000  15.000000  Default  30.0000  30.0000
195            municipality   6.000000   6.000000  Default  29.0000  29.0000
180                   local  16.000000  16.000000  Default  28.0000  28.0000
194               municipal   6.000000   6.000000  Default  27.0000  27.0000
20                     anti   3.000000   3.000000  Default  26.0000  26.0000
..                      ...        ...        ...      ...      ...      ...
64                  conduct   0.159299   2.797575   Topic5  -5.8646   0.4338
315              transition   0.159279   2.944768   Topic5  -5.8647   0.3824
149       increase capacity   0.159261   2.898094   Topic5  -5.8648   0.3982
128  government institution   0.159261   2.776205   Topic5  -5.8648   0.4412
240          project design   0.159242   3.534076   Topic5  -5.8649   0.1997

[274 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         1  0.485842  accountable
3         3  0.485842  accountable
3         4  0.242921  accountable
5         2  0.412595       action
5         4  0.412595       action
...     ...       ...          ...
322       1  0.486012         work
322       2  0.364509         work
322       3  0.121503         work
323       1  0.370764         year
323       2  0.370764         year

[419 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 3, 4])

In [15]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,0.202504,1.586671,2.542276,0.200032,1.741101,1.590602,1.833569,2.749435,1.977885,2.036993,...,0.713383,1.286885,3.771677,2.050433,0.460387,0.201620,1.189196,3.498592,3.409494,1.016194
Topic1,0.604088,0.358838,2.078744,2.006376,0.724133,0.200057,0.823681,0.200120,1.000716,0.303334,...,0.348964,0.482001,2.232186,1.467583,1.882662,0.549105,0.203697,0.204888,1.626201,0.319559
Topic2,0.818088,0.946758,0.205438,0.733649,0.201238,0.948764,0.852293,0.551620,0.311484,0.875236,...,2.821309,0.662541,0.456654,0.201883,0.441406,1.251290,0.498901,0.201610,0.579071,0.561587
Topic3,0.200175,0.200202,0.200151,0.200159,0.200162,0.200231,0.200170,0.200478,0.200214,0.200189,...,0.200173,0.200342,0.465376,0.200165,0.200173,0.200184,0.494299,0.200152,0.200170,0.565309
Topic4,1.650193,3.677835,4.292236,1.945505,1.835483,0.216745,3.847302,0.270529,2.506735,2.818848,...,4.800841,1.047011,4.415278,1.592234,3.562627,0.747403,2.919776,1.122903,4.179554,0.901282


In [16]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,corruption,government,trade,public,reduce,project,improve,program,procurement,reform,economic,information,design,transparency,institution
Topic 1,municipality,local,municipal,participation,citizen,government,service,local government,governance,program,citizen participation,capacity,increase,democratic,decision
Topic 2,corruption,anti,anti corruption,education,technical assistance,technical,legal,assistance,provide technical assistance,project,provide technical,training,awareness,program,provide
Topic 3,corruption,report,encourage,ensure,year,non,governmental,anticorruption,use,citizen,objective,transparency,organization,focus,program
Topic 4,project,strengthen,local,justice,support,reform,improve,judicial,aim,government,program,capacity,public,increase,development
